In [9]:
import torch
import os
import torchvision.models as models

from torchvision import datasets

import numpy as np
from glob import glob

from PIL import Image

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torchvision.transforms as transforms
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

In [ ]:
files = np.array(glob("./256x256_NormalizedWhaleImages/*"))

In [10]:
#Preprocessing (will add test images later) (Augmentation will be removed later once run with entire dataset)
preprocess = {
    '256x256_NormalizedWhaleImages': transforms.Compose([
        transforms.RandomResizedCrop(224), #Tensor size
        transforms.RandomHorizontalFlip(), #Augment the data to work with a small data set size (100 right now)
        transforms.RandomRotation(15), #Augment the data
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
    ]),
    
    'test_images_100': transforms.Compose([
        transforms.Resize(256), #Resize to image size
        transforms.CenterCrop(224), #Tensor size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
        
    ])
    
}

image_path = "./"

#Load the images
images = {i: datasets.ImageFolder(os.path.join(image_path, i), preprocess[i])
          for i in ['256x256_NormalizedWhaleImages']}
dataloaders = {x: torch.utils.data.DataLoader(images[x], batch_size = 20, shuffle = True, num_workers = 0) 
                for x in ['256x256_NormalizedWhaleImages']}

In [11]:
#Making the CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        #Layer 1
        self.layer1 =  nn.Sequential(
            nn.Conv2d(3, 16, 3, padding = 1),
            nn.BatchNorm2d(224, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        #Layer 2
        self.layer2 =  nn.Sequential(
            nn.Conv2d(16, 32, 3, padding = 1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        #Layer 3
        self.layer3 =  nn.Sequential(
            nn.Conv2d(32, 64, 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        #Layer 4
        self.layer4 =  nn.Sequential(
            nn.Conv2d(64, 128, 3, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        #Layer 5
        self.layer5 =  nn.Sequential(
            nn.Conv2d(128, 256, 3, padding = 1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2)
            
        )
        
        self.final_bn = nn.BatchNorm2d(256)
        
        #FC Layer
        self.drop = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(256*7*7, 512)
        self.fc2 = nn.Linear(512, len(images['256x256_NormalizedWhaleImages'].classes))

In [12]:
#Forward function
def forward(self, x):
    logits = self.layer1(x)
    logits = self.layer2(logits)
    logits = self.layer3(logits)
    logits = self.layer4(logits)
    logits = self.layer5(logits)
    
    logits = self.final_bn(logits)
    
    logits = self.drop(logits)
    
    logits = self.fc1(logits)
    logits = self.drop(logits)
    logits = self.fc2(logits)
    
    return logits

In [ ]:
#Instantiating a CNN
